In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import io
import boto3

import s3fs
fs = s3fs.S3FileSystem(anon=False,key='######',secret='########')

from dotenv import load_dotenv
load_dotenv(verbose=True)

def aws_session(region_name='us-east-1'):
    return boto3.session.Session(aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'), #looks for any .env file
                                aws_secret_access_key=os.getenv('AWS_ACCESS_KEY_SECRET'), #Has to be in same directory
                                region_name=region_name) #from above

def make_bucket(name, acl): 
    session = aws_session()
    s3_resource = session.resource('s3')
    return s3_resource.create_bucket(Bucket=name, ACL=acl)

def upload_file_to_bucket(bucket_name, file_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    file_dir, file_name = os.path.split(file_path)

    bucket = s3_resource.Bucket(bucket_name)
    bucket.upload_file(
      Filename=file_path,
      Key=file_name,
      ExtraArgs={'ACL': 'public-read'}
    )

    s3_url = f"https://{bucket_name}.s3.amazonaws.com/{file_name}"
    return s3_url

def download_file_from_bucket(bucket_name, s3_key, dst_path):
    session = aws_session()
    s3_resource = session.resource('s3')
    bucket = s3_resource.Bucket(bucket_name)
    bucket.download_file(Key=s3_key, Filename=dst_path)

## download_file_from_bucket('music-demo-lyrics', 'lyrics_25k.csv', 'short_name.csv')
## with open('short_name.csv') as fo:
    ## print(fo.read())

In [2]:
download_file_from_bucket('worm-begin', 'lyrics_25k.csv', 'lyrics_25k_df.csv')
with open('lyrics_25k_df.csv') as fo:
    lyrics_25k_df = pd.read_csv(fo)
    
lyrics_25k_df

,Unnamed: 0,link,artist,song_name,lyrics
0,0,../lyrics/backstreetboys/climbingthewalls.html,Backstreet Boys Lyrics,Climbing The Walls,"\n\n[Brian:]\nClose your eyes, make a wish\nTh..."
1,1,../lyrics/westlife/howtobreakaheart.html,Westlife Lyrics,How To Break A Heart,\n\nSince you're not worth my love\nI haven't ...
2,2,../lyrics/deanmartin/iwill.html,Dean Martin Lyrics,I Will,\n\nI don't want to be the one to say I'm gonn...
3,3,../lyrics/deanmartin/tellyourmotherhello.html,Dean Martin Lyrics,Tell Your Mother Hello,\n\nSure I loved the dear money that shines in...
4,4,../lyrics/deanmartin/behonestwithme.html,Dean Martin Lyrics,Be Honest With Me,\n\nBe honest with me dear whatever you do\nRe...
...,...,...,...,...,...
25737,25737,../lyrics/jasonmraz/whatmamasay.html,Jason Mraz Lyrics,What Mama Say,\n\n(La la da da da da...)\n\nWhatever mama sa...
25738,25738,https://www.azlyrics.com/lyrics/gesaffelstein/...,Gesaffelstein Lyrics,Blast Off,\n\nYou've been thinking about this all night\...
25739,25739,../lyrics/jasonmraz/aworldwithyou.html,Jason Mraz Lyrics,A World With You,\n\nLet's hit the road and throw out the map\n...
25740,25740,../lyrics/jasonmraz/loveisreal.html,Jason Mraz Lyrics,Love Is Real,\n\nLove is real it is not just in novels or t...


In [3]:
lyrics_25k2_df =  pd.DataFrame((lyrics_25k_df), columns=['original_csv','artist','song_name','link','lyrics','language','genre','date'])
lyrics_25k2_df

,original_csv,artist,song_name,link,lyrics,language,genre,date
0,NaN,Backstreet Boys Lyrics,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,NaN
1,NaN,Westlife Lyrics,How To Break A Heart,../lyrics/westlife/howtobreakaheart.html,\n\nSince you're not worth my love\nI haven't ...,NaN,NaN,NaN
2,NaN,Dean Martin Lyrics,I Will,../lyrics/deanmartin/iwill.html,\n\nI don't want to be the one to say I'm gonn...,NaN,NaN,NaN
3,NaN,Dean Martin Lyrics,Tell Your Mother Hello,../lyrics/deanmartin/tellyourmotherhello.html,\n\nSure I loved the dear money that shines in...,NaN,NaN,NaN
4,NaN,Dean Martin Lyrics,Be Honest With Me,../lyrics/deanmartin/behonestwithme.html,\n\nBe honest with me dear whatever you do\nRe...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
25737,NaN,Jason Mraz Lyrics,What Mama Say,../lyrics/jasonmraz/whatmamasay.html,\n\n(La la da da da da...)\n\nWhatever mama sa...,NaN,NaN,NaN
25738,NaN,Gesaffelstein Lyrics,Blast Off,https://www.azlyrics.com/lyrics/gesaffelstein/...,\n\nYou've been thinking about this all night\...,NaN,NaN,NaN
25739,NaN,Jason Mraz Lyrics,A World With You,../lyrics/jasonmraz/aworldwithyou.html,\n\nLet's hit the road and throw out the map\n...,NaN,NaN,NaN
25740,NaN,Jason Mraz Lyrics,Love Is Real,../lyrics/jasonmraz/loveisreal.html,\n\nLove is real it is not just in novels or t...,NaN,NaN,NaN


In [4]:
lyrics_25k2_df['artist'].replace(('Lyrics'), '', regex = True, inplace = True)
lyrics_25k2_df

,original_csv,artist,song_name,link,lyrics,language,genre,date
0,NaN,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,NaN
1,NaN,Westlife,How To Break A Heart,../lyrics/westlife/howtobreakaheart.html,\n\nSince you're not worth my love\nI haven't ...,NaN,NaN,NaN
2,NaN,Dean Martin,I Will,../lyrics/deanmartin/iwill.html,\n\nI don't want to be the one to say I'm gonn...,NaN,NaN,NaN
3,NaN,Dean Martin,Tell Your Mother Hello,../lyrics/deanmartin/tellyourmotherhello.html,\n\nSure I loved the dear money that shines in...,NaN,NaN,NaN
4,NaN,Dean Martin,Be Honest With Me,../lyrics/deanmartin/behonestwithme.html,\n\nBe honest with me dear whatever you do\nRe...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
25737,NaN,Jason Mraz,What Mama Say,../lyrics/jasonmraz/whatmamasay.html,\n\n(La la da da da da...)\n\nWhatever mama sa...,NaN,NaN,NaN
25738,NaN,Gesaffelstein,Blast Off,https://www.azlyrics.com/lyrics/gesaffelstein/...,\n\nYou've been thinking about this all night\...,NaN,NaN,NaN
25739,NaN,Jason Mraz,A World With You,../lyrics/jasonmraz/aworldwithyou.html,\n\nLet's hit the road and throw out the map\n...,NaN,NaN,NaN
25740,NaN,Jason Mraz,Love Is Real,../lyrics/jasonmraz/loveisreal.html,\n\nLove is real it is not just in novels or t...,NaN,NaN,NaN


In [5]:
lyrics_25k2_df['original_csv'] = '25K'
lyrics_25k2_df

,original_csv,artist,song_name,link,lyrics,language,genre,date
0,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,NaN
1,25K,Westlife,How To Break A Heart,../lyrics/westlife/howtobreakaheart.html,\n\nSince you're not worth my love\nI haven't ...,NaN,NaN,NaN
2,25K,Dean Martin,I Will,../lyrics/deanmartin/iwill.html,\n\nI don't want to be the one to say I'm gonn...,NaN,NaN,NaN
3,25K,Dean Martin,Tell Your Mother Hello,../lyrics/deanmartin/tellyourmotherhello.html,\n\nSure I loved the dear money that shines in...,NaN,NaN,NaN
4,25K,Dean Martin,Be Honest With Me,../lyrics/deanmartin/behonestwithme.html,\n\nBe honest with me dear whatever you do\nRe...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
25737,25K,Jason Mraz,What Mama Say,../lyrics/jasonmraz/whatmamasay.html,\n\n(La la da da da da...)\n\nWhatever mama sa...,NaN,NaN,NaN
25738,25K,Gesaffelstein,Blast Off,https://www.azlyrics.com/lyrics/gesaffelstein/...,\n\nYou've been thinking about this all night\...,NaN,NaN,NaN
25739,25K,Jason Mraz,A World With You,../lyrics/jasonmraz/aworldwithyou.html,\n\nLet's hit the road and throw out the map\n...,NaN,NaN,NaN
25740,25K,Jason Mraz,Love Is Real,../lyrics/jasonmraz/loveisreal.html,\n\nLove is real it is not just in novels or t...,NaN,NaN,NaN


In [6]:
lyrics_25k2_df =  pd.DataFrame((lyrics_25k_df), columns=['original_csv','artist','song_name','link','lyrics','language','genre'])

In [7]:
lyrics_25k2_df

,original_csv,artist,song_name,link,lyrics,language,genre
0,NaN,Backstreet Boys Lyrics,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN
1,NaN,Westlife Lyrics,How To Break A Heart,../lyrics/westlife/howtobreakaheart.html,\n\nSince you're not worth my love\nI haven't ...,NaN,NaN
2,NaN,Dean Martin Lyrics,I Will,../lyrics/deanmartin/iwill.html,\n\nI don't want to be the one to say I'm gonn...,NaN,NaN
3,NaN,Dean Martin Lyrics,Tell Your Mother Hello,../lyrics/deanmartin/tellyourmotherhello.html,\n\nSure I loved the dear money that shines in...,NaN,NaN
4,NaN,Dean Martin Lyrics,Be Honest With Me,../lyrics/deanmartin/behonestwithme.html,\n\nBe honest with me dear whatever you do\nRe...,NaN,NaN
...,...,...,...,...,...,...,...
25737,NaN,Jason Mraz Lyrics,What Mama Say,../lyrics/jasonmraz/whatmamasay.html,\n\n(La la da da da da...)\n\nWhatever mama sa...,NaN,NaN
25738,NaN,Gesaffelstein Lyrics,Blast Off,https://www.azlyrics.com/lyrics/gesaffelstein/...,\n\nYou've been thinking about this all night\...,NaN,NaN
25739,NaN,Jason Mraz Lyrics,A World With You,../lyrics/jasonmraz/aworldwithyou.html,\n\nLet's hit the road and throw out the map\n...,NaN,NaN
25740,NaN,Jason Mraz Lyrics,Love Is Real,../lyrics/jasonmraz/loveisreal.html,\n\nLove is real it is not just in novels or t...,NaN,NaN


In [9]:
lyrics_25k3_df = lyrics_25k2_df.rename(columns={'artist':'artist_name'})
lyrics_25k3_df

,original_csv,artist_name,song_name,link,lyrics,language,genre
0,NaN,Backstreet Boys Lyrics,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN
1,NaN,Westlife Lyrics,How To Break A Heart,../lyrics/westlife/howtobreakaheart.html,\n\nSince you're not worth my love\nI haven't ...,NaN,NaN
2,NaN,Dean Martin Lyrics,I Will,../lyrics/deanmartin/iwill.html,\n\nI don't want to be the one to say I'm gonn...,NaN,NaN
3,NaN,Dean Martin Lyrics,Tell Your Mother Hello,../lyrics/deanmartin/tellyourmotherhello.html,\n\nSure I loved the dear money that shines in...,NaN,NaN
4,NaN,Dean Martin Lyrics,Be Honest With Me,../lyrics/deanmartin/behonestwithme.html,\n\nBe honest with me dear whatever you do\nRe...,NaN,NaN
...,...,...,...,...,...,...,...
25737,NaN,Jason Mraz Lyrics,What Mama Say,../lyrics/jasonmraz/whatmamasay.html,\n\n(La la da da da da...)\n\nWhatever mama sa...,NaN,NaN
25738,NaN,Gesaffelstein Lyrics,Blast Off,https://www.azlyrics.com/lyrics/gesaffelstein/...,\n\nYou've been thinking about this all night\...,NaN,NaN
25739,NaN,Jason Mraz Lyrics,A World With You,../lyrics/jasonmraz/aworldwithyou.html,\n\nLet's hit the road and throw out the map\n...,NaN,NaN
25740,NaN,Jason Mraz Lyrics,Love Is Real,../lyrics/jasonmraz/loveisreal.html,\n\nLove is real it is not just in novels or t...,NaN,NaN


"""Why is 'Lyrics' back again?"""

In [10]:
lyrics_25k3_df['artist_name'].replace(('Lyrics'), '', regex = True, inplace = True)
lyrics_25k3_df

,original_csv,artist_name,song_name,link,lyrics,language,genre
0,NaN,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN
1,NaN,Westlife,How To Break A Heart,../lyrics/westlife/howtobreakaheart.html,\n\nSince you're not worth my love\nI haven't ...,NaN,NaN
2,NaN,Dean Martin,I Will,../lyrics/deanmartin/iwill.html,\n\nI don't want to be the one to say I'm gonn...,NaN,NaN
3,NaN,Dean Martin,Tell Your Mother Hello,../lyrics/deanmartin/tellyourmotherhello.html,\n\nSure I loved the dear money that shines in...,NaN,NaN
4,NaN,Dean Martin,Be Honest With Me,../lyrics/deanmartin/behonestwithme.html,\n\nBe honest with me dear whatever you do\nRe...,NaN,NaN
...,...,...,...,...,...,...,...
25737,NaN,Jason Mraz,What Mama Say,../lyrics/jasonmraz/whatmamasay.html,\n\n(La la da da da da...)\n\nWhatever mama sa...,NaN,NaN
25738,NaN,Gesaffelstein,Blast Off,https://www.azlyrics.com/lyrics/gesaffelstein/...,\n\nYou've been thinking about this all night\...,NaN,NaN
25739,NaN,Jason Mraz,A World With You,../lyrics/jasonmraz/aworldwithyou.html,\n\nLet's hit the road and throw out the map\n...,NaN,NaN
25740,NaN,Jason Mraz,Love Is Real,../lyrics/jasonmraz/loveisreal.html,\n\nLove is real it is not just in novels or t...,NaN,NaN


In [11]:
lyrics_25k3_df

,original_csv,artist_name,song_name,link,lyrics,language,genre
0,NaN,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN
1,NaN,Westlife,How To Break A Heart,../lyrics/westlife/howtobreakaheart.html,\n\nSince you're not worth my love\nI haven't ...,NaN,NaN
2,NaN,Dean Martin,I Will,../lyrics/deanmartin/iwill.html,\n\nI don't want to be the one to say I'm gonn...,NaN,NaN
3,NaN,Dean Martin,Tell Your Mother Hello,../lyrics/deanmartin/tellyourmotherhello.html,\n\nSure I loved the dear money that shines in...,NaN,NaN
4,NaN,Dean Martin,Be Honest With Me,../lyrics/deanmartin/behonestwithme.html,\n\nBe honest with me dear whatever you do\nRe...,NaN,NaN
...,...,...,...,...,...,...,...
25737,NaN,Jason Mraz,What Mama Say,../lyrics/jasonmraz/whatmamasay.html,\n\n(La la da da da da...)\n\nWhatever mama sa...,NaN,NaN
25738,NaN,Gesaffelstein,Blast Off,https://www.azlyrics.com/lyrics/gesaffelstein/...,\n\nYou've been thinking about this all night\...,NaN,NaN
25739,NaN,Jason Mraz,A World With You,../lyrics/jasonmraz/aworldwithyou.html,\n\nLet's hit the road and throw out the map\n...,NaN,NaN
25740,NaN,Jason Mraz,Love Is Real,../lyrics/jasonmraz/loveisreal.html,\n\nLove is real it is not just in novels or t...,NaN,NaN


In [13]:
download_file_from_bucket('worm-begin', 'album_details_25k.csv', 'year_df.csv')
year_df = pd.read_csv('year_df.csv')

"""Strip the  'Lyrics' off of every 'singer_name', reduce to just what I need
to pd-merge and the year column I want to add to the main csv"""

year_df['singer_name'].replace(('Lyrics'), '', regex = True, inplace = True)
year2_df = pd.DataFrame((year_df), columns=['singer_name','year'])
year2_df=year2_df.rename(columns={'singer_name':'artist_name'})

In [14]:
year2_df

,artist_name,year
0,Taylor Swift,2006
1,Taylor Swift,2007
2,Taylor Swift,2008
3,Taylor Swift,2010
4,Taylor Swift,2012
...,...,...
1814,Jason Mraz,2012
1815,Jason Mraz,2014
1816,Jason Mraz,2018
1817,Jason Mraz,2020


In [15]:
Main_25K_df = pd.merge(lyrics_25k3_df, year2_df, on='artist_name')

In [16]:
Main_25K_df

,original_csv,artist_name,song_name,link,lyrics,language,genre,year
0,NaN,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1995
1,NaN,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1997
2,NaN,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1999
3,NaN,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2000
4,NaN,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2005
...,...,...,...,...,...,...,...,...
512664,NaN,Noah Cyrus,July,../lyrics/noahcyrus/july.html,\n\nI've been holding my breath\nI've been cou...,NaN,NaN,2020
512665,NaN,Noah Cyrus,July,../lyrics/noahcyrus/july.html,\n\nI've been holding my breath\nI've been cou...,NaN,NaN,Not Defined
512666,NaN,Noah Cyrus,Stay Together,../lyrics/noahcyrus/staytogether.html,\n\nI drank straight to my head\nI went outsid...,NaN,NaN,2018
512667,NaN,Noah Cyrus,Stay Together,../lyrics/noahcyrus/staytogether.html,\n\nI drank straight to my head\nI went outsid...,NaN,NaN,2020


In [18]:
Main_25K_df['original_csv'] = '25K'
Main_25K_df
""" The Big K """

,original_csv,artist_name,song_name,link,lyrics,language,genre,year
0,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1995
1,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1997
2,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,1999
3,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2000
4,25K,Backstreet Boys,Climbing The Walls,../lyrics/backstreetboys/climbingthewalls.html,"\n\n[Brian:]\nClose your eyes, make a wish\nTh...",NaN,NaN,2005
...,...,...,...,...,...,...,...,...
512664,25K,Noah Cyrus,July,../lyrics/noahcyrus/july.html,\n\nI've been holding my breath\nI've been cou...,NaN,NaN,2020
512665,25K,Noah Cyrus,July,../lyrics/noahcyrus/july.html,\n\nI've been holding my breath\nI've been cou...,NaN,NaN,Not Defined
512666,25K,Noah Cyrus,Stay Together,../lyrics/noahcyrus/staytogether.html,\n\nI drank straight to my head\nI went outsid...,NaN,NaN,2018
512667,25K,Noah Cyrus,Stay Together,../lyrics/noahcyrus/staytogether.html,\n\nI drank straight to my head\nI went outsid...,NaN,NaN,2020


In [19]:
Main_25K_df.to_csv('Main_25k.csv', index= False)
upload_file_to_bucket('music-demo-lyrics','Main_25k_df.csv')

'https://music-demo-lyrics.s3.amazonaws.com/Main_25k_df.csv'